# ==================================================================================
# TECH CHALLENGE FASE 3 - TRATAMENTO DE DADOS
# ==================================================================================


In [ ]:
# ==================================================================================
# IMPORTAÇÕES E CONFIGURAÇÕES
# ==================================================================================
import pandas as pd
import numpy as np
import warnings
import os

warnings.filterwarnings('ignore')

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("="*80)
print("TRATAMENTO DE DADOS - CRIAÇÃO DO FLIGHTS_CLEAN.CSV")
print("="*80)


## 1. CARREGAMENTO DOS DADOS


In [ ]:

df_flights = pd.read_csv('../data/flights.csv')

print(f"\n✓ Dados carregados com sucesso!")
print(f"  - Flights: {df_flights.shape}")
print(f"  - Memória utilizada: {df_flights.memory_usage(deep=True).sum() / 1024**3:.2f} GB")
print(f"\nPrimeiras linhas:")
df_flights.head()


## 2. ANÁLISE DE VALORES AUSENTES (MISSING VALUES)


In [ ]:

missing_data = pd.DataFrame({
    'Coluna': df_flights.columns,
    'Missing_Count': df_flights.isnull().sum(),
    'Missing_Percent': (df_flights.isnull().sum() / len(df_flights)) * 100
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percent', ascending=False)

print("\n" + "-"*80)
print("2.1 COLUNAS COM VALORES AUSENTES")
print("-"*80)
if len(missing_data) > 0:
    print(missing_data.to_string(index=False))
else:
    print("✓ Nenhuma coluna com valores ausentes!")

## 3. TRATAMENTO DE VALORES AUSENTES


In [ ]:
print(f"Total de colunas com missing values: {len(missing_data)}")
print(f"\nDetalhamento por categoria:")
print(f"  • Colunas com >60% missing: {len(missing_data[missing_data['Missing_Percent'] > 60])}")
print(f"  • Colunas com 1-60% missing: {len(missing_data[(missing_data['Missing_Percent'] > 1) & (missing_data['Missing_Percent'] <= 60)])}")
print(f"  • Colunas com <1% missing: {len(missing_data[missing_data['Missing_Percent'] < 1])}")

print("\nColunas com missing values:")
for idx, row in missing_data.iterrows():
    print(f"  • {row['Coluna']}: {row['Missing_Count']:,} ({row['Missing_Percent']:.2f}%)")


df_flights_clean = df_flights.copy()


### 3.1 Tratando Delays Específicos


In [ ]:
delay_columns = ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 
                 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']

for col in delay_columns:
    missing_count = df_flights_clean[col].isnull().sum()
    if missing_count > 0:
        df_flights_clean[col] = df_flights_clean[col].fillna(0)
        print(f"  ✓ {col}: {missing_count:,} valores preenchidos com 0")


### 3.2 Tratando Colunas de Voos Cancelados


In [ ]:
cancel_related_cols = ['DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
                       'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'WHEELS_ON', 'TAXI_IN']

for col in cancel_related_cols:
    missing_count = df_flights_clean[col].isnull().sum()
    if missing_count > 0:
        treated_count = 0
        
        if 'DELAY' in col:
            
            mask = (df_flights_clean['CANCELLED'] == 0) & (df_flights_clean[col].isnull())
            treated_count = mask.sum()
            df_flights_clean.loc[mask, col] = 0
            print(f"  ✓ {col}: {treated_count:,} valores preenchidos com 0 (voos não cancelados)")
        elif col in ['TAXI_OUT', 'TAXI_IN']:
            median_val = df_flights_clean[df_flights_clean['CANCELLED'] == 0][col].median()
            mask = (df_flights_clean['CANCELLED'] == 0) & (df_flights_clean[col].isnull())
            treated_count = mask.sum()
            df_flights_clean.loc[mask, col] = median_val
            print(f"  ✓ {col}: {treated_count:,} valores preenchidos com mediana ({median_val:.2f})")
        else:
            treated_count = 0
            print(f"  → {col}: {missing_count:,} valores mantidos como NaN (voos cancelados/desviados)")
        
        if treated_count > 0:
            remaining = missing_count - treated_count
            if remaining > 0:
                print(f"    → {remaining:,} valores mantidos como NaN (voos cancelados)")


### 3.3 Tratando Colunas com Poucos Missing Values


In [ ]:

for col in ['AIR_TIME', 'ELAPSED_TIME']:
    missing_count = df_flights_clean[col].isnull().sum()
    if missing_count > 0:
        
        median_val = df_flights_clean[df_flights_clean['CANCELLED'] == 0][col].median()
        mask = (df_flights_clean['CANCELLED'] == 0) & (df_flights_clean[col].isnull())
        df_flights_clean.loc[mask, col] = median_val
        print(f"  ✓ {col}: {mask.sum():,} valores preenchidos com mediana ({median_val:.2f})")

if df_flights_clean['TAIL_NUMBER'].isnull().sum() > 0:
    missing_count = df_flights_clean['TAIL_NUMBER'].isnull().sum()
    df_flights_clean['TAIL_NUMBER'] = df_flights_clean['TAIL_NUMBER'].fillna('UNKNOWN')
    print(f"  ✓ TAIL_NUMBER: {missing_count:,} valores preenchidos com 'UNKNOWN'")

if df_flights_clean['SCHEDULED_TIME'].isnull().sum() > 0:
    missing_count = df_flights_clean['SCHEDULED_TIME'].isnull().sum()
    median_val = df_flights_clean['SCHEDULED_TIME'].median()
    df_flights_clean['SCHEDULED_TIME'] = df_flights_clean['SCHEDULED_TIME'].fillna(median_val)
    print(f"  ✓ SCHEDULED_TIME: {missing_count:,} valores preenchidos com mediana ({median_val:.2f})")


## 4. VERIFICAÇÃO APÓS TRATAMENTO


In [ ]:
missing_after = df_flights_clean.isnull().sum()
missing_after = missing_after[missing_after > 0].sort_values(ascending=False)

if len(missing_after) > 0:
    print("\nColunas que ainda possuem missing values (estruturais):")
    for col, count in missing_after.items():
        pct = (count / len(df_flights_clean)) * 100
        print(f"  • {col}: {count:,} ({pct:.2f}%)")
    print("\n  → Estes são missing values estruturais (voos cancelados/desviados)")
    print("    e devem ser mantidos como NaN para preservar a informação.")
else:
    print("  ✓ Nenhum missing value restante!")

print(f"Total de missing values antes: {df_flights.isnull().sum().sum():,}")
print(f"Total de missing values depois: {df_flights_clean.isnull().sum().sum():,}")
print(f"Redução: {df_flights.isnull().sum().sum() - df_flights_clean.isnull().sum().sum():,} valores tratados")


## 5. Salvando Dataset


In [ ]:
output_path = '../data/flights_clean.csv'
df_flights_clean.to_csv(output_path, index=False)
print(f"✓ Dataset limpo salvo em: {output_path}")
print(f"  - Registros originais: {len(df_flights):,}")
print(f"  - Registros após tratamento: {len(df_flights_clean):,}")
print(f"  - Colunas: {df_flights_clean.shape[1]}")

# Verificar se o arquivo foi criado
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / (1024**2)  # Tamanho em MB
    print(f"  - Tamanho do arquivo: {file_size:.2f} MB")
    print("\n✓ Arquivo flights_clean.csv criado com sucesso!")
else:
    print("\n✗ Erro ao criar o arquivo!")


## 6. RESUMO DO TRATAMENTO

Este notebook executou o tratamento de dados necessário para criar o arquivo `flights_clean.csv`. As principais ações realizadas foram:

1. **Delays Específicos**: Preenchidos com 0 (missing = não houve atraso desse tipo)
2. **Colunas de Voos Cancelados**: 
   - Delays preenchidos com 0 para voos não cancelados
   - Taxi times preenchidos com mediana para voos não cancelados
   - Outras colunas mantidas como NaN para voos cancelados (informação estrutural)
3. **CANCELLATION_REASON**: Mantido como está (missing = voo não cancelado)
4. **Colunas com Poucos Missing**: 
   - AIR_TIME e ELAPSED_TIME: preenchidos com mediana
   - TAIL_NUMBER: preenchido com 'UNKNOWN'
   - SCHEDULED_TIME: preenchido com mediana

O dataset limpo preserva os missing values estruturais (voos cancelados/desviados) que são importantes para análise, mas trata os valores ausentes que podem ser imputados de forma segura.
